In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f'{current_date=}')

current_date=Timestamp('2024-01-30 06:00:00')


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2024-01-02 06:00:00 to 2024-01-30 05:00:00
Finished: Reading data from Hopsworks, using ArrowFlight (7.05s) 


In [5]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] bagging_fraction is set=0.5959370704450638, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5959370704450638


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [6]:
predictions

,pickup_location_id,predicted_demand
0,1,1.0
1,2,0.0
2,3,0.0
3,4,3.0
4,5,0.0
...,...,...
252,261,4.0
253,262,115.0
254,263,91.0
255,264,41.0


In [7]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2024-01-30 06:00:00
1,2,0.0,2024-01-30 06:00:00
2,3,0.0,2024-01-30 06:00:00
3,4,3.0,2024-01-30 06:00:00
4,5,0.0,2024-01-30 06:00:00
...,...,...,...
252,261,4.0,2024-01-30 06:00:00
253,262,115.0,2024-01-30 06:00:00
254,263,91.0,2024-01-30 06:00:00
255,264,41.0,2024-01-30 06:00:00


In [12]:
import src.config as config
import hopsworks
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generated by our production model",
    primary_key = ['pickup_location_id','pickup_hour'],
    event_time = 'pickup_hour'
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job":False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/423065/fs/418888/fg/472450


Uploading Dataframe: 0.00% |          | Rows 0/257 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/423065/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1e7caff0d00>, None)